# 1. Merging Raw Data from Evolutionary Simulations

**Imports**

In [4]:
import numpy as np

import sympy as sym

import itertools

import pandas as pd

import glob

In [5]:
from operator import itemgetter

In [6]:
import matplotlib.pyplot as plt

import matplotlib

from mpl_toolkits.axes_grid1 import make_axes_locatable

In [7]:
import tqdm

## Preparation to Read Files

In [8]:
cs = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

In [9]:
names1 = [f"p_{i}" for i in range(1, 3)] + ["t", "cooperation"]
names2 = [f"p_{i}" for i in range(1, 5)] + ["t", "cooperation"]
names3 = [f"p_{i}" for i in range(1, 9)] + ["t", "cooperation"]

In [11]:
def frequencies(df, max_steps=10**7):
    frequencies = []
    previous = 0
    for value in df["t"][1:]:
        frequencies.append(value - previous)
        previous = value
    frequencies.append(max_steps - previous)

    df["frequencies"] = np.array(frequencies)
    df["frequencies norm"] = np.array(frequencies) / max_steps

    return df

We want to assess how many resident strategies are partner strategies in our simulation. For a resident strategy to be classified as a partner, it must satisfy all inequalities in the respective definition of partner strategies and cooperate
with a probability of at least 95\% after full cooperation.

In [12]:
def one_bit_condition_one(p1, p2, c, b):
    return p2 < 1 - c / b


def one_bit_condition_two(p1, p2, c, b):
    return p1 > 0.95


def is_partner_one_bit(p1, p2, c, b):
    return (one_bit_condition_one(p1, p2, c, 1)) and (
        one_bit_condition_two(p1, p2, c, 1)
    )

In [13]:
def two_bit_condition_one(p1, p2, p3, p4, c, b):
    return p4 < (1 - c / b)


def two_bit_condition_two(p1, p2, p3, p4, c, b):
    return p2 + p3 < (2 - c / b)


def two_bit_condition_three(p1, p2, p3, p4, c, b):
    return p1 > 0.95


def is_partner_two_bits(p1, p2, p3, p4, c, b):
    return (
        (two_bit_condition_one(p1, p2, p3, p4, c, 1))
        and two_bit_condition_two(p1, p2, p3, p4, c, 1)
        and two_bit_condition_three(p1, p2, p3, p4, c, 1)
    )

In [14]:
def three_bit_condition_one(p1, p2, p3, p4, p5, p6, p7, p8, c, b):
    return p8 < (1 - c / b)


def three_bit_condition_two(p1, p2, p3, p4, p5, p6, p7, p8, c, b):
    return p3 + p6 < (2 - c / b)


def three_bit_condition_three(p1, p2, p3, p4, p5, p6, p7, p8, c, b):
    return p2 + p3 + p5 < (3 - c / b)


def three_bit_condition_four(p1, p2, p3, p4, p5, p6, p7, p8, c, b):
    return p4 + p6 + p7 < (3 - 2 * c / b)


def three_bit_condition_five(p1, p2, p3, p4, p5, p6, p7, p8, c, b):
    return p2 + p4 + p5 + p7 < (4 - 2 * c / b)


def three_bit_condition_six(p1, p2, p3, p4, p5, p6, p7, p8, c, b):
    return p1 > 0.95


def is_partner_three_bits(p1, p2, p3, p4, p5, p6, p7, p8, c, b):
    return (
        (three_bit_condition_one(p1, p2, p3, p4, p5, p6, p7, p8, c, 1))
        and (three_bit_condition_two(p1, p2, p3, p4, p5, p6, p7, p8, c, 1))
        and (three_bit_condition_three(p1, p2, p3, p4, p5, p6, p7, p8, c, 1))
        and (three_bit_condition_four(p1, p2, p3, p4, p5, p6, p7, p8, c, 1))
        and (three_bit_condition_five(p1, p2, p3, p4, p5, p6, p7, p8, c, 1))
        and (three_bit_condition_six(p1, p2, p3, p4, p5, p6, p7, p8, c, 1))
    )

#  Reactive-1

In [20]:
files = glob.glob(f"../data/matlab_data/bits_2_beta_*_seed_1_*.csv")

files += glob.glob(f"../data/beta_2/bits_2_beta_*_seed_1_*.csv")

In [23]:
one_bits = []

for file in files:
    c = float(file.split("c_")[1].split(".csv")[0])

    beta = float(file.split("_beta_")[1].split("_")[0])

    df = pd.read_csv(file, names=names1)

    df = df.drop_duplicates()

    df["c"] = c

    df["beta"] = beta

    df = frequencies(df, 10**7)

    one_bits.append(df)

In [24]:
one_bits = pd.concat(one_bits)

In [25]:
one_bits["beta"].unique()

array([10. ,  1. ,  0.1,  2. ])

In [26]:
ispartner = []

for i, row in tqdm.tqdm(one_bits.iterrows()):
    condition = is_partner_one_bit(row["p_1"], row["p_2"], row["c"], 1)

    ispartner.append(condition)

1668615it [00:58, 28498.26it/s]


In [27]:
one_bits["isPartner"] = ispartner

In [29]:
one_bits.to_csv("../prc_data/evolution_over_c_and_beta_reactive_one.csv")

# Reactive-1

In [35]:
files = glob.glob(f"../data/matlab_data/bits_4_beta_0.1_seed_1_*.csv")

files += glob.glob(f"../data/matlab_data/bits_4_beta_1_seed_1_*.csv")

files += glob.glob(f"../data/beta_2/bits_4_beta_2_seed_1_*.csv")

two_bits = []

for file in files:
    c = float(file.split("c_")[1].split(".csv")[0])

    beta = float(file.split("_beta_")[1].split("_")[0])

    df = pd.read_csv(file, names=names2)

    df = df.drop_duplicates()

    df["c"] = c

    df["beta"] = beta

    df = frequencies(df, 10**7)

    two_bits.append(df)

In [36]:
files = glob.glob("../data/long/bits_4_beta_10_seed_1_c_*.csv")

for file in files:
    c = float(file.split("c_")[1].split(".csv")[0])

    beta = float(file.split("beta_")[1].split("_")[0])

    df = pd.read_csv(file, names=names2)

    df = df.drop_duplicates()

    df["c"] = c

    df["beta"] = beta

    df = frequencies(df, 10**8)

    two_bits.append(df)

In [37]:
two_bits = pd.concat(two_bits)

In [38]:
two_bits["beta"].unique()

array([ 0.1,  1. ,  2. , 10. ])

In [39]:
ispartner = []

for i, row in tqdm.tqdm(two_bits.iterrows()):
    condition = is_partner_two_bits(
        row["p_1"], row["p_2"], row["p_3"], row["p_4"], row["c"], 1
    )

    ispartner.append(condition)

1913480it [01:14, 25775.21it/s]


In [40]:
two_bits["isPartner"] = ispartner

In [41]:
two_bits.to_csv("../prc_data/evolution_over_c_and_beta_reactive_two.csv")

## Reactive-2 Counting

In [53]:
files = glob.glob(f"../data/counting/bits_4_beta_0.1_seed_1_c_*.csv")
files += glob.glob(f"../data/counting/bits_4_beta_1_seed_1_c_*.csv")
files += glob.glob(f"../data/counting/bits_4_beta_10_seed_1_c_*.csv")
files += glob.glob(f"../data/counting/bits_4_beta_2_seed_1_c_*.csv")

In [54]:
counting_two_bits = []

for file in files:
    c = float(file.split("c_")[1].split(".csv")[0])

    beta = float(file.split("beta_")[1].split("_")[0])

    df = pd.read_csv(file, names=names2)

    df = df.drop_duplicates()

    df["c"] = c

    df["beta"] = beta

    df = frequencies(df, 10**7)

    counting_two_bits.append(df)

In [55]:
counting_two_bits = pd.concat(counting_two_bits)

In [56]:
ispartner = []

for i, row in tqdm.tqdm(counting_two_bits.iterrows()):
    condition = is_partner_two_bits(
        row["p_1"], row["p_2"], row["p_3"], row["p_4"], row["c"], 1
    )

    ispartner.append(condition)

1886774it [01:14, 25240.79it/s]


In [57]:
counting_two_bits["isPartner"] = ispartner

In [58]:
counting_two_bits["beta"].unique()

array([ 0.1,  1. , 10. ,  2. ])

In [59]:
counting_two_bits.to_csv(
    "../prc_data/evolution_over_c_and_beta_reactive_counting_two.csv"
)

# Reactive-3

In [60]:
files = glob.glob(f"../data/matlab_data/bits_8_beta_0.1_seed_1_*.csv")
files += glob.glob(f"../data/matlab_data/bits_8_beta_1_seed_1_*.csv")
files += glob.glob(f"../data/beta_2/bits_8_beta_2_seed_1_*.csv")

In [61]:
three_bits = []

for file in files:
    c = float(file.split("c_")[1].split(".csv")[0])

    beta = float(file.split("_beta_")[1].split("_")[0])

    df = pd.read_csv(file, names=names3)

    df = df.drop_duplicates()

    df["c"] = c

    df["beta"] = beta

    df = frequencies(df, 2 * 10**7)

    three_bits.append(df)

In [62]:
files = glob.glob(f"../data/beta_2/bits_8_beta_10_seed_1_c_*.csv")

In [63]:
for file in files:
    c = float(file.split("c_")[1].split(".csv")[0])

    beta = float(file.split("_beta_")[1].split("_")[0])

    df = pd.read_csv(file, names=names3)

    df = df.drop_duplicates()

    df["c"] = c

    df["beta"] = beta

    df = frequencies(df, 40000000)

    three_bits.append(df)

In [64]:
three_bits = pd.concat(three_bits)

In [65]:
three_bits["beta"].unique()

array([ 0.1,  1. ,  2. , 10. ])

In [66]:
ispartner = []

for i, row in tqdm.tqdm(three_bits.iterrows()):
    condition = is_partner_three_bits(
        row["p_1"],
        row["p_2"],
        row["p_3"],
        row["p_4"],
        row["p_5"],
        row["p_6"],
        row["p_7"],
        row["p_8"],
        row["c"],
        1,
    )

    ispartner.append(condition)

2640162it [02:05, 21047.36it/s]


In [67]:
three_bits["isPartner"] = ispartner

In [68]:
three_bits.to_csv("../prc_data/evolution_over_c_and_beta_reactive_three.csv")

## Reactive-3 Counting

In [70]:
files = glob.glob(f"../data/counting/bits_8_beta_0.1_seed_1_c_*.csv")
files += glob.glob(f"../data/counting/bits_8_beta_1_seed_1_c_*.csv")
files += glob.glob(f"../data/counting/bits_8_beta_10_seed_1_c_*.csv")
files += glob.glob(f"../data/counting/bits_8_beta_2_seed_1_c_*.csv")

In [71]:
counting_three_bits = []

for file in files:
    c = float(file.split("c_")[1].split(".csv")[0])

    beta = float(file.split("beta_")[1].split("_")[0])

    df = pd.read_csv(file, names=names3)

    df = df.drop_duplicates()

    df["c"] = c

    df["beta"] = beta

    df = frequencies(df, 2 * 10**7)

    counting_three_bits.append(df)

In [72]:
counting_three_bits = pd.concat(counting_three_bits)

In [73]:
ispartner = []

for i, row in tqdm.tqdm(counting_three_bits.iterrows()):
    condition = is_partner_three_bits(
        row["p_1"],
        row["p_2"],
        row["p_3"],
        row["p_4"],
        row["p_5"],
        row["p_6"],
        row["p_7"],
        row["p_8"],
        row["c"],
        1,
    )

    ispartner.append(condition)

3580753it [03:00, 19822.09it/s]


In [74]:
counting_three_bits["isPartner"] = ispartner

In [75]:
counting_three_bits["beta"].unique()

array([ 0.1,  1. , 10. ,  2. ])

In [76]:
counting_three_bits.to_csv(
    "../prc_data/evolution_over_c_and_beta_reactive_counting_three.csv"
)

# Results with different seeds

In [2]:
import os

In [3]:
import glob

In [21]:
dfs = []

for sdim, type_, max_step, name in zip(
    [2, 4, 8],
    ["one", "two", "three"],
    [10**7, 10**7, 2 * 10**7],
    [names1, names2, names3],
):
    for seed in range(11, 21):
        file = f"../data/evolution_over_seeds/bits_{sdim}_beta_1_seed_{seed}_c_0.5.csv"

        df = pd.read_csv(file, names=name)

        df = df.drop_duplicates()

        df["type"] = type_
        df["seed"] = seed

        df = frequencies(df, max_step)

        dfs.append(df)

        os.remove(file)

In [22]:
df = pd.concat(dfs)

In [25]:
df1 = pd.read_csv("../prc_data/evolution_over_seeds_reactive.csv", index_col=0)

In [34]:
df1['seed'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [35]:
df['seed'].unique()

array([11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

In [42]:
merged = pd.concat([df, df1])

In [51]:
merged = merged.sort_values(['type', 'seed'])

In [52]:
merged.to_csv("../prc_data/evolution_over_seeds_reactive_new.csv")

## Results for Counting Strategies

In [15]:
dfs = []

for sdim, type_, max_step, name in zip(
    [4, 8],
    ["counting-two", "counting-three"],
    [10**7, 2 * 10**7],
    [names2, names3],
):
    for seed in range(1, 11):
        file = f"../data/counting/bits_{sdim}_beta_1_seed_{seed}_c_0.5.csv"

        df = pd.read_csv(file, names=name)

        df = df.drop_duplicates()

        df["type"] = type_
        df["seed"] = seed

        df = frequencies(df, max_step)

        dfs.append(df)

In [16]:
df = pd.concat(dfs)

In [17]:
dfs = []

for sdim, type_, max_step, name in zip(
    [8],
    ["counting-three"],
    [10**7],
    [names3],
):
    for seed in range(11, 21):
        file = f"../data/counting/bits_{sdim}_beta_1_seed_{seed}_c_0.5.csv"

        df = pd.read_csv(file, names=name)

        df = df.drop_duplicates()

        df["type"] = type_
        df["seed"] = seed

        df = frequencies(df, max_step)

        dfs.append(df)

In [19]:
len(dfs)

10

In [23]:
df2 = pd.concat(dfs)

In [25]:
df = pd.concat([df, df2])

In [28]:
three = df[df['type'] == 'counting-three']

In [29]:
groups = three.groupby('seed')

In [46]:
top = []

for i, group in groups:
    top.append(group.sort_values("frequencies").iloc[-1][['p_1', 'p_2', 'p_7', 'p_8']])

In [47]:
[np.mean([t[i] for t in top]) for i in range(4)]

[0.9997153739, 0.30753884539000004, 0.4504472216, 0.21307141803000001]

In [48]:
three

,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,t,cooperation,type,seed,frequencies,frequencies norm
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,counting-three,20,1918,1.918000e-04
1,0.094599,0.539822,0.539822,0.784930,0.539822,0.784930,0.784930,0.092454,1918,0.488276,counting-three,20,72,7.200000e-06
2,0.082884,0.028336,0.028336,0.515928,0.028336,0.515928,0.515928,0.609005,1990,0.378122,counting-three,20,39,3.900000e-06
3,0.006741,0.068689,0.068689,0.636416,0.068689,0.636416,0.636416,0.185294,2029,0.337690,counting-three,20,277,2.770000e-05
4,0.929925,0.522713,0.522713,0.020979,0.522713,0.020979,0.020979,0.378545,2306,0.289447,counting-three,20,9,9.000000e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47899,0.012864,0.641242,0.641242,0.024589,0.641242,0.024589,0.024589,0.553392,9999845,0.317108,counting-three,20,13,1.300000e-06
47900,0.817132,0.607970,0.607970,0.183911,0.607970,0.183911,0.183911,0.115471,9999858,0.219667,counting-three,20,9,9.000000e-07
47901,0.665477,0.662695,0.662695,0.478390,0.662695,0.478390,0.478390,0.568924,9999867,0.605439,counting-three,20,8,8.000000e-07
47902,0.625830,0.382034,0.382034,0.445524,0.382034,0.445524,0.445524,0.251547,9999875,0.395384,counting-three,20,8,8.000000e-07


In [85]:
df.to_csv("../prc_data/evolution_over_seeds_reactive_counting.csv")